<img src= "A2.png">

## Trabalho Final da Disciplina "Redes Neurais e Algoritmos Genéticos": Construção de dietas usando a inteligência artificial

### Contextualização:

Durante esse trabalho temos como objetivo principal por em prática, propondo uma solução para um problema cientificamente relevante, tudo o conhecimento adquirido ao longo da disciplina de Redes Neurais e Algoritmos Géneticos ministrada pelo Profº Drº Daniel Cassar no 3º semestre do Bacharelado em Ciência e Tecnologia da Ilum Escola de Ciência, faculdade do Centro Nacional de Pesquisa em Energia e Materiais(CNPEM), centro esse que abriga também o Sirius, maior e mais avançado acelerador de partículas da América Latina e do Hemisfério Sul.

### Introdução:

O problema sobre o qual nos debruçamos trata-se de um problema de otimização, onde queremos que o nosso paciente tenha uma dieta com a melhor nutrição(a ser avaliada pela densidade nutricional dos alimentos) para um determinado limite calórico(de acordo com o objetivo dele). Dessa forma, podemos perceber que se trata de um problema de maximização com restrição e decidimos, baseados nos conhecimentos que adquirimos ao longo do semestre utilizar como algoritmo base o algoritmos genético que implementa uma solução para o problema da mochila com restrição; cujo exemplo de implementação e uso pode ser visto em  - [experimento A.07 - aplicando restricoes.ipynb](https://github.com/Sarah-Freire/RNAG/blob/main/AlgoritmosGeneticos/experimento%20A.07%20-%20aplicando%20restricoes.ipynb).

### Importações:

In [1]:
#Importações
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


import random

from funcoes import computa_cesta
from funcoes import funcao_objetivo_pop_dieta
from funcoes import populacao_cesta_alimento as cria_populacao_inicial
from funcoes import selecao_roleta_max as funcao_selecao
from funcoes import cruzamento_ponto_simples as funcao_cruzamento
from funcoes import mutacao_dieta as funcao_mutacao

### Obtenção dos Dados do Usuário e do seu Objetivo

Para começarmos, então, faz-se necessários que conversemos com nosso usuário para a obtenção de seus dados e o estabelecimento dos seus objetivos de modo a proporcionarmos uma resposta mais personalizada e otimizada.

In [2]:
dados = []
def informacoes_da_pessoa():

    advertencia = ["Digite uma resposta válida"]
    

    print("Qual é o seu nome?")
    nome = input()
    dados.append(nome)

    print("Qual a sua idade?(em anos)")
    idade_str = input()
    idade = float(idade_str)
    if idade in range(18,120):
        dados.append(idade) 
    else:
        print(advertencia)
        idade = input()

    print("Qual o seu peso?(em quilogramas)")
    peso_str = input()
    peso = float(peso_str)
    if peso == round(peso, 2):
        dados.append(peso) 
    else:
        print(advertencia)
        peso = input()

    print("Qual é o seu sexo?")
    sexo = input()
    while sexo.lower() not in ["feminino","masculino"]:
        print(advertencia)
        sexo = input()
    dados.append(sexo)

    print("Qual é a sua altura?(em metros)")
    altura_str = input()
    altura = float(altura_str)
    if altura == round(altura, 2):
        dados.append(altura) 
    else:
        print(advertencia)
        altura = input()

    print("Qual é o seu nível de atividade física?\n sedentário: pouco ou nenhum exercício físico\n pouco ativo:leve 1-3/semana\n moderadamente ativo: moderado 3-5/semana\n muito ativo: pesado 7-7 dias/semana\n extremamente ativo: pesado/intenso ou treino 2xdia")
    atividade = input()
    while atividade.lower() not in ["sedentário", "pouco ativo", "moderadamente ativo", "muito ativo", "extremamente ativo"]:
        print(advertencia)
        atividade = input()
    dados.append(atividade)
    
    print("Qual é o seu objetivo com a dieta?\n manter massa\n perder massa\n ganhar massa")
    objetivo = input()
    while objetivo.lower() not in ["manter massa", "perder massa", "ganhar massa"]:
        print(advertencia)
        objetivo = input()
    dados.append(objetivo)
    
    return dados

print(informacoes_da_pessoa())

Qual é o seu nome?


 vitor


Qual a sua idade?(em anos)


 19


Qual o seu peso?(em quilogramas)


...

Qual é o seu objetivo com a dieta?
 manter massa
 perder massa
 ganhar massa


 ganhar mass


['Digite uma resposta válida']


 ganhar massa


['vitor', 19.0, 75.5, 'masculino', 1.83, 'moderadamente ativo', 'ganhar massa']


#### A partir disso como determinaremos a quantidade de calorias ideais?

A equação de Harris-Benedict é uma fórmula que utiliza a altura, peso, idade e sexo de determinado indivíduo para calcular a sua taxa metabólica basal(TMB)

**1º Passo:**
Determinar a taxa metabólica basal (TMB)

Homens: TMB = 66 + (13.7 x peso em quilogramas) + (5 x altura em centímetros) – (6.8 x idade em anos)

Mulheres: TMB = 65.5 + (9.6 X peso em quilogramas) + (1.8 X altura em centímetros) – (4.7 x idade em anos).

**2º Passo:**
Multiplicar o valor da TMB pelo fator de atividade física adequado

- Sedentário – pouco ou nenhum exercício (TMB x 1.2)

- Pouco ativo – exercício/esporte leve 1-3 dias/semana (TMB x 1.375)

- Moderadamente ativo – exercício/esporte moderado 3-5 dias/semana (TMB x 1.55)

- Muito ativo – exercício/esporte pesado 6-7 dias/semana (TMB x 1.725)

- Extremamente ativo – exercício/esporte muito pesado e trabalho físico intenso diariamente ou treino de 2x ao dia (TMB x 1.9)

Este cálculo vai te dar um valor estimado das necessidades energéticas diárias de uma pessoa. Contudo, não se esqueça que se trata apenas de uma estimativa e que deve ser usada como um guia e não como uma regra.

In [8]:
#armazenando dados nos varávies
altura = dados[4]
peso = dados[2]
idade = dados[1]
sexo = dados[3]
atividade = dados[5]
objetivo = dados[6]

In [9]:
def tmb_func(altura, peso, idade, sexo, atividade,objetivo):
    if sexo == "feminino":
        tmb_1 = 655.1 + (9.6 * peso) + (1.8 * altura) - (4.7 * idade)
    else:
        tmb_1= 66.5 + (13.7 * peso) + (5 * altura) - (6.8 * idade)
        
    if atividade == "sedentário":
        tmb = tmb_1 * 1.2
    elif atividade == "pouco ativo":
        tmb = tmb_1 * 1.375
    elif atividade == "moderadamente ativo":
        tmb = tmb_1 * 1.55
    elif atividade == "muito ativo":
        tmb = tmb_1 * 1.725
    else:
        tmb = tmb_1 * 1.9
        
    if objetivo.lower() == "manter massa":
        
        tmb_final = tmb
        
    elif objetivo.lower() == "perder massa":
        
        tmb_final = tmb - 500
        
    else:
        
        tmb_final = tmb + 500
        
    return tmb_final

print(tmb_func(altura,peso,idade,sexo, atividade, objetivo))

2020.24


#### Agora que temos os dados necessários e a nossa função definida, podemos, enfim, começar a criar o nosso código do nutrella

In [10]:
### CONSTANTES

# relacionadas à busca
TAMANHO_POP = 20
NUM_GERACOES = 100
CHANCE_CRUZAMENTO = 0.5
CHANCE_MUTACAO = 0.1

# relacionadas ao problema a ser resolvido
# LIMITE_DE_CALORIAS = tmb_func(altura,peso,idade,sexo, atividade, objetivo)
LIMITE_DE_CALORIAS = tmb_func(altura,peso,idade,sexo, atividade, objetivo)


# O supermercado

# Leia o arquivo Excel como um DataFrame
df_alimentos = pd.read_excel("Alimentos.xlsx")

#Agora precisamos transformar esses df em dicionários(com orientação por linhas);
dic_alimentos = df_alimentos.to_dict(orient='records')

df_alimentosC = pd.read_excel("Alimentos - Calorias.xlsx")
dic_alimentosC = df_alimentosC.to_dict(orient='records')

SUPERMERCADO = {}
for item in dic_alimentosC:
    alimento = item['Alimentos']
    calorias_dic = item['Energia(Kcal)']
    Densidade_nutricional = item['Densidade nutricional']
    SUPERMERCADO[alimento] = {'calorias': calorias_dic, 'Densidade nutricional': Densidade_nutricional}
    
NUM_ALIMENTOS = len(SUPERMERCADO)
ORDEM_DOS_NOMES = list(sorted(SUPERMERCADO.keys()))

In [11]:
# Funções locais
print(LIMITE_DE_CALORIAS)
def funcao_objetivo_pop(populacao):
    return funcao_objetivo_pop_dieta(populacao, SUPERMERCADO, LIMITE_DE_CALORIAS, ORDEM_DOS_NOMES)

2020.24


In [7]:
# Busca por algoritmo genético
populacao = cria_populacao_inicial(TAMANHO_POP, NUM_ALIMENTOS)

# variaveis para o hall da fama
melhor_fitness_ja_visto = -float("inf") # levando em consideração que é um problema de maximização
melhor_individuo_ja_visto = [0] * NUM_ALIMENTOS # solução trivial para o problema

for n in range(NUM_GERACOES):

    # Seleção
    fitness = funcao_objetivo_pop(populacao)
    populacao = funcao_selecao(populacao, fitness)

    # Cruzamento
    pais = populacao[0::2]
    maes = populacao[1::2]

    contador = 0

    for pai, mae in zip(pais, maes):
        if random.random() <= CHANCE_CRUZAMENTO:
            filho1, filho2 = funcao_cruzamento(pai, mae)
            populacao[contador] = filho1
            populacao[contador + 1] = filho2

        contador = contador + 2

    # Mutação
    for n in range(len(populacao)):
        if random.random() <= CHANCE_MUTACAO:
            individuo = populacao[n]
            populacao[n] = funcao_mutacao(individuo)

    # melhor individuo já visto até agora (hall da fama)
    fitness = funcao_objetivo_pop(populacao)
    maior_fitness = max(fitness)
    posicao = fitness.index(maior_fitness)
    individuo = populacao[posicao].copy()
    densidade_nutri_alimento, calorias = computa_cesta(individuo, SUPERMERCADO, ORDEM_DOS_NOMES)
    if maior_fitness > melhor_fitness_ja_visto : # não mostrar coisas q n resolvem o problema
        melhor_fitness_ja_visto = maior_fitness
        melhor_individuo_ja_visto = individuo
        print(f"Maior densidade nutricional: {densidade_nutri_alimento} | Calorias: {calorias}")

#Resultado
print()
print()
print("Oi,",dados[0], "aqui estão os itens que você deve pegar:")
for pega_ou_nao, item in zip(melhor_individuo_ja_visto, ORDEM_DOS_NOMES):
    if pega_ou_nao == 1:
        print("+", item)
print()
valor_total, peso_total = computa_cesta(
    melhor_individuo_ja_visto, SUPERMERCADO, ORDEM_DOS_NOMES
)
valor_total_arredondado = round(valor_total, 0)
peso_total_arredondado = round(peso_total, 0)
print(
    f"Com isso, sua mochila terá o valor nutricional de {valor_total_arredondado}"
    f" e quantidade calóricade de {peso_total_arredondado} kcal."
)

Maior densidade nutricional: 147.55416302294378 | Calorias: 1507.5300000000002
Maior densidade nutricional: 150.9235591091491 | Calorias: 1438.56
Maior densidade nutricional: 153.49372952029802 | Calorias: 2019.0600000000002
Maior densidade nutricional: 161.76768866380496 | Calorias: 1685.56
Maior densidade nutricional: 163.48699439851234 | Calorias: 1879.4499999999998
Maior densidade nutricional: 177.76525526807757 | Calorias: 1925.4499999999998
Maior densidade nutricional: 178.92382363559813 | Calorias: 1833.37
Maior densidade nutricional: 189.22652641011368 | Calorias: 1928.0900000000001
Maior densidade nutricional: 193.07170930941328 | Calorias: 1920.09
Maior densidade nutricional: 200.54852492284013 | Calorias: 1789.0900000000001
Maior densidade nutricional: 200.6365056924256 | Calorias: 1996.3899999999999
Maior densidade nutricional: 213.9793388763285 | Calorias: 1875.0900000000001
Maior densidade nutricional: 214.24852492284012 | Calorias: 1807.0900000000001
Maior densidade nutr

### 

### Referências:

[1] https://www.omnicalculator.com/health/bmr-harris-benedict-equation

[2] https://en.wikipedia.org/wiki/Harris%E2%80%93Benedict_equation

[3] https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5074671/

[4] https://www.sanarmed.com/equacao-de-harris-benedict-colunistas